# Lecture 2 - Fourier reconstruction

# Contents

* The Fourier-Slice Theorem
* Fourier reconstruction

# The Fourier-Slice Theorem

The Fourier-Slice Theorem relates the 2D-Fourier transform of $u$ to the 1D-Fourier transform of $f$:

$$\widehat{f}(\sigma,\theta) = \widehat{u}(|\sigma| \mathbf{n}_\theta),$$

with

$$\widehat{f}(\sigma,\theta) = \int_{\mathbb{R}} f(s,\theta) e^{-2\pi\imath \sigma s}\mathrm{d}s,$$

$$\widehat{u}(\mathbf{k}) = \int_{\mathbb{R}^2} u(\mathbf{x}) e^{-2\pi\imath \mathbf{k}\cdot \mathbf{x}}\mathrm{d}\mathbf{x},$$

$$\mathbf{n}_\theta = (\cos\theta,\sin\theta).$$

We can visualise this as follows

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/97/Radon_transform_via_Fourier_transform.png/2000px-Radon_transform_via_Fourier_transform.png)

# Fourier reconstruction

* In principle, we can reconstruct the image by performing an inverse Fourier transform
* Having collected *discrete* measurements $f_{ij} = f(s_i,\theta_j)$, we only have partial information on the Fourier transform of $u$
* Ultimately, we will represent $u$ on a grid of pixels with intenties $u_i = u(\mathbf{x}_i)$

## The Discrete Fourier Transform

The *discrete Fourier transform (DFT)* of a sequence $\{g_i\}_{i=0}^n$ is defined as

$$\widehat{g}_i = \sum_{j=0}^{n-1} g_j \exp\left(-\frac{2\pi\imath}{n} ij\right), \quad i = 0, \ldots, n-1.$$

If $g_i = g(i\cdot\Delta x)$, then $\widehat{g}_i = \widehat{g}( \ldots )$.

In matrix-vector notation, we express this as

$$\widehat{\mathbf{g}} = F\mathbf{g},$$

with $F_{ij} = \exp\left(-\frac{2\pi\imath}{n} ij\right)$.

The inverse is given by

$$F^{-1} = n^{-1} F^*.$$

Be aware that some interesting things may happen with the DFT, when subsampling or truncating the signals.

## The Fast Fourier Transform

A naive implementation would require $\mathcal{O}(n^2)$ operations to multiply by $F$. With some clever tricks, however, it can be done in $\mathcal{O}(n\log n)$.

## Interpolation

* Applying the Fourier transform to $f_{ij}$ gives us samples of $\widehat{u}$ at wavenumbers $\{(\sigma_i \mathbf{n}_{\theta_j})\}$, with $\sigma_i = \ldots$ (assuming again that $f_{ij} = f(i \cdot \Delta s ,
\theta_j)$)
* To usefully apply the inverse DFT, we need $\widehat{u}$ at wavenumbers $\{(\ldots, \ldots)\}$.

A linear interpolation scheme ...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk
from skimage.transform import radon
from scipy.fft import fft2, ifft2, fft, ifft, fftfreq, fftshift
from scipy.interpolate import LinearNDInterpolator

# settings
nx = 32
na = 32
theta = np.linspace(0., 180., na)
s = np.linspace(-1,1,nx)

# phantom
ii,jj = disk((nx//2+nx//8,nx//2+nx//8),0.2 * (nx//2))

u = np.zeros((nx,nx))
u[ii,jj] = 1

# sinogram
f = radon(u, theta=theta)

# fourier transform of sinogram
fh = fft(fftshift(f,axes=0),axis=0)
sigma = fftfreq(nx,s[1]-s[0])
kk1 = np.outer(sigma,np.cos(np.pi*theta/180))
kk2 = np.outer(sigma,-np.sin(np.pi*theta/180))

# target grid
k = fftfreq(nx,(s[1]-s[0]))
kt1,kt2 = np.meshgrid(k,k)

# interpolation
uh = LinearNDInterpolator(np.stack((kk1.ravel(),kk2.ravel()),axis=1),fh.ravel(),fill_value=0)(kt1,kt2)
#uh = griddata((kk1.ravel(),kk2.ravel()),fh.ravel(),(kt1.ravel(),kt2.ravel()),fill_value=0)]

# ifft
ur = np.real(fftshift(ifft2(uh.reshape(nx,nx))))

In [ ]:
fig,ax = plt.subplots(1,3)

ax[0].plot(kk1.ravel(),kk2.ravel(),'.',alpha=.1)
ax[0].plot(kt1.ravel(),kt2.ravel(),'r.',alpha=.1)
ax[0].set_aspect(1)

ax[1].scatter(kk1.ravel(),kk2.ravel(),c=np.abs(fh.ravel()))
ax[1].set_aspect(1)

ax[2].imshow(np.abs(fftshift(uh.reshape(nx,nx))),extent=(-max(k),max(k),-max(k),max(k)))
ax[2].set_aspect(1)

fig.tight_layout()
fig.set_figwidth(10)

In [ ]:
fig,ax = plt.subplots(1,2)

ax[0].imshow(u,extent=(-1,1,-1,1))
ax[1].imshow(ur,extent=(-1,1,-1,1))

Representing the sinogram and image as vectors, the image recontruction step can be represented as 

$$\mathbf{u} = (F^{-1} \otimes F^{-1}) T (I \otimes F) \mathbf{f}.$$

In Python, we would implement this as 

```python
u .. 
```

## The Non-Uniform Fourier Transform

We can avoid the interpolation step by applying a *non-uniform FFT*.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.draw import disk
from skimage.transform import radon
from scipy.fft import fft2, ifft2, fft, ifft, fftfreq, fftshift
from scipy.interpolate import LinearNDInterpolator

from pynufft import NUFFT

# settings
nx = 32
na = 32
theta = np.linspace(0., 180., na)
s = np.linspace(-1,1,nx)

# phantom
ii,jj = disk((nx//2+nx//8,nx//2+nx//8),0.2 * (nx//2))

u = np.zeros((nx,nx))
u[ii,jj] = 1

# sinogram
f = radon(u, theta=theta)

# fourier transform of sinogram
fh = fft(fftshift(f,axes=0),axis=0)
sigma = fftfreq(nx,s[1]-s[0])
kk1 = np.outer(sigma,np.cos(np.pi*theta/180))
kk2 = np.outer(sigma,-np.sin(np.pi*theta/180))

# target grid
k = fftfreq(nx,(s[1]-s[0]))
kt1,kt2 = np.meshgrid(k,k)

# interpolation
uh = LinearNDInterpolator(np.stack((kk1.ravel(),kk2.ravel()),axis=1),fh.ravel(),fill_value=0)(kt1,kt2)
#uh = griddata((kk1.ravel(),kk2.ravel()),fh.ravel(),(kt1.ravel(),kt2.ravel()),fill_value=0)]

# ifft
ur = np.real(fftshift(ifft2(uh.reshape(nx,nx))))

/opt/anaconda3/lib/python3.8/site-packages/skimage/transform/radon_transform.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  coords = np.array(np.ogrid[:image.shape[0], :image.shape[1]])
